In [1]:
from chgk_rating_client import Rating
from collections import defaultdict
from pprint import pprint
from typing import Dict, List, NamedTuple

In [2]:
class Question(NamedTuple):
    number: int
    teams: int
    percentage: float
        
    def __repr__(self):
        return f"{self.number:2} — {self.teams:2} команд ({self.percentage:2.2f}%)"


def build_question_mapping(tournament_results: List) -> Dict:
    qs = defaultdict(int)
    for team in tournament_results:
        mask_fixed = team["mask"].replace("?", "0")
        for q_number, q_result in enumerate(mask_fixed, 1):
            qs[q_number] += int(q_result)
    return qs

            
def get_team_stats(team_id: int, tournament_id: int, client: Rating) -> List[Question]:
    all_results = client.tournament_results(tournament_id)
    teams_count = len(all_results)
    questions = build_question_mapping(all_results)
    
    team_results = [r for r in all_results if int(r['idteam']) == team_id][0]
    team_zeroes = [q_num + 1 
                   for q_num, q_res in enumerate(team_results['mask']) 
                   if q_res != '1']
    
    stats = [Question(q_number, questions[q_number], round(questions[q_number] / teams_count * 100.0, 2)) 
             for q_number in team_zeroes]
    
    return sorted(stats, key=lambda x: -x.teams)

In [3]:
rating_client = Rating(redis_host="localhost")

In [4]:
tournament_id = 5795
team_id = 27129
stats = get_team_stats(team_id, tournament_id, rating_client)
for s in stats:
    print(s)

31 — 73 команд (74.49%)
27 — 44 команд (44.90%)
45 — 39 команд (39.80%)
22 — 31 команд (31.63%)
13 — 28 команд (28.57%)
43 — 28 команд (28.57%)
52 — 27 команд (27.55%)
35 — 20 команд (20.41%)
 6 — 19 команд (19.39%)
34 — 19 команд (19.39%)
 7 — 17 команд (17.35%)
29 — 15 команд (15.31%)
46 — 14 команд (14.29%)
56 — 11 команд (11.22%)
19 — 10 команд (10.20%)
26 — 10 команд (10.20%)
10 —  8 команд (8.16%)
55 —  8 команд (8.16%)
60 —  6 команд (6.12%)
57 —  5 команд (5.10%)
42 —  4 команд (4.08%)
50 —  4 команд (4.08%)
36 —  2 команд (2.04%)
25 —  1 команд (1.02%)
 8 —  0 команд (0.00%)
 9 —  0 команд (0.00%)
44 —  0 команд (0.00%)
51 —  0 команд (0.00%)
